In [1]:
%run ../../common_notebooks/setup_spark_connection.ipynb

In [2]:
def get_spark_catalog_objects(spark):
    # List databases, tables, and functions in the Spark catalog
    databases = spark.catalog.listDatabases()
    tables = spark.catalog.listTables()
    functions = spark.catalog.listFunctions()

    print("Databases:", [db.name for db in databases])
    print("Tables:", [t.name for t in tables])
    print("Functions:", [f.name for f in functions])

In [3]:
get_spark_catalog_objects(spark)

Databases: ['default']
Tables: []
Functions: ['!', '!=', '%', '&', '*', '+', '-', '/', '<', '<<', '<=', '<=>', '<>', '=', '==', '>', '>=', '>>', '>>>', '^', 'abs', 'acos', 'acosh', 'add_months', 'aes_decrypt', 'aes_encrypt', 'aggregate', 'and', 'any', 'any_value', 'approx_count_distinct', 'approx_percentile', 'array', 'array_agg', 'array_append', 'array_compact', 'array_contains', 'array_distinct', 'array_except', 'array_insert', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_prepend', 'array_remove', 'array_repeat', 'array_size', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'ascii', 'asin', 'asinh', 'assert_true', 'atan', 'atan2', 'atanh', 'avg', 'base64', 'between', 'bigint', 'bin', 'binary', 'bit_and', 'bit_count', 'bit_get', 'bit_length', 'bit_or', 'bit_xor', 'bitmap_bit_position', 'bitmap_bucket_number', 'bitmap_construct_agg', 'bitmap_count', 'bitmap_or_agg', 'bool_and', 'bool_or', 'boolean', 'bround', 'btrim', 'cardinality', '

In [4]:
# Check if database exists and drop it
if spark.catalog.databaseExists("db01"):
    spark.sql("DROP DATABASE db01 CASCADE")

In [5]:
get_spark_catalog_objects(spark)

Databases: ['default']
Tables: []
Functions: ['!', '!=', '%', '&', '*', '+', '-', '/', '<', '<<', '<=', '<=>', '<>', '=', '==', '>', '>=', '>>', '>>>', '^', 'abs', 'acos', 'acosh', 'add_months', 'aes_decrypt', 'aes_encrypt', 'aggregate', 'and', 'any', 'any_value', 'approx_count_distinct', 'approx_percentile', 'array', 'array_agg', 'array_append', 'array_compact', 'array_contains', 'array_distinct', 'array_except', 'array_insert', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_prepend', 'array_remove', 'array_repeat', 'array_size', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'ascii', 'asin', 'asinh', 'assert_true', 'atan', 'atan2', 'atanh', 'avg', 'base64', 'between', 'bigint', 'bin', 'binary', 'bit_and', 'bit_count', 'bit_get', 'bit_length', 'bit_or', 'bit_xor', 'bitmap_bit_position', 'bitmap_bucket_number', 'bitmap_construct_agg', 'bitmap_count', 'bitmap_or_agg', 'bool_and', 'bool_or', 'boolean', 'bround', 'btrim', 'cardinality', '

In [6]:
spark.sql("CREATE DATABASE IF NOT EXISTS db01")
spark.sql("USE db01")

DataFrame[]

In [7]:
spark.sql("""
CREATE TABLE IF NOT EXISTS db01.table1 (
    id INT,
    name STRING,
    age INT
) USING delta
""")

# Insert data
spark.sql("""
INSERT INTO db01.table1 VALUES
    (1, 'Alice', 30),
    (2, 'Bob', 25)
""")

DataFrame[]

In [8]:
# Query the table
spark.sql("SELECT * FROM db01.table1").show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 30|
|  2|  Bob| 25|
+---+-----+---+



In [9]:
# Update data
spark.sql("UPDATE db01.table1 SET age = age + 1 WHERE id = 1")

# Query after update
spark.sql("SELECT * FROM db01.table1").show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 31|
|  2|  Bob| 25|
+---+-----+---+



In [10]:
spark.sql("DROP TABLE IF EXISTS bikeSharing")
spark.sql("DROP TABLE IF EXISTS bikeSharingpartition")

DataFrame[]

In [11]:
spark.sql("""
CREATE TABLE IF NOT EXISTS bikeSharing (
    instant INT,
    dteday DATE,
    season INT,
    yr INT
) USING DELTA
LOCATION 'bikeSharing'
""")

DataFrame[]

In [12]:
# Create a DataFrame with sample data for bikeSharing table
from pyspark.sql import Row
from datetime import date

data = [
    Row(instant=1, dteday=date(2023, 1, 1), season=1, yr=0),
    Row(instant=2, dteday=date(2023, 1, 2), season=1, yr=0),
    Row(instant=3, dteday=date(2023, 4, 15), season=2, yr=0),
    Row(instant=4, dteday=date(2023, 7, 20), season=3, yr=1),
    Row(instant=5, dteday=date(2023, 10, 10), season=4, yr=1)
]

df = spark.createDataFrame(data)
df.write.format("delta").mode("overwrite").save("/tmp/delta-table/bikeSharing")

In [13]:
spark.sql("""
CREATE TABLE IF NOT EXISTS bikeSharingPartition (
    instant INT,
    dteday DATE,
    season INT,
    yr INT
) USING DELTA
PARTITIONED BY (season)
LOCATION '/tmp/delta-table/bikeSharingPartition'
""")

DataFrame[]

In [14]:
# Insert data from bikeSharing into bikeSharingPartition
df_to_insert = spark.sql("SELECT * FROM bikeSharing")
df_to_insert.write.format("delta").mode("append").save("/tmp/delta-table/bikeSharingPartition")

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `db01`.`bikesharing` doesn't exist.

JVM stacktrace:
org.apache.spark.sql.delta.DeltaAnalysisException
	at org.apache.spark.sql.delta.DeltaErrorsBase.nonExistentDeltaTable(DeltaErrors.scala:1570)
	at org.apache.spark.sql.delta.DeltaErrorsBase.nonExistentDeltaTable$(DeltaErrors.scala:1569)
	at org.apache.spark.sql.delta.DeltaErrors$.nonExistentDeltaTable(DeltaErrors.scala:3759)
	at org.apache.spark.sql.delta.catalog.DeltaTableV2.toBaseRelation$lzycompute(DeltaTableV2.scala:297)
	at org.apache.spark.sql.delta.catalog.DeltaTableV2.toBaseRelation(DeltaTableV2.scala:285)
	at org.apache.spark.sql.delta.DeltaRelation$.$anonfun$fromV2Relation$1(DeltaAnalysis.scala:1380)
	at org.apache.spark.sql.delta.metering.DeltaLogging.recordFrameProfile(DeltaLogging.scala:171)
	at org.apache.spark.sql.delta.metering.DeltaLogging.recordFrameProfile$(DeltaLogging.scala:169)
	at org.apache.spark.sql.delta.DeltaRelation$.recordFrameProfile(DeltaAnalysis.sc

In [15]:
# Verify the data was inserted
spark.sql("SELECT * FROM bikeSharingPartition").show()

+-------+------+------+---+
|instant|dteday|season| yr|
+-------+------+------+---+
+-------+------+------+---+

